# Goals

- read a file from s3 or local
- data analysis
    - lower case
    - date correct format
- charting
- export data analysis

In [ ]:
#run to install modules
#pip install awswrangler pandas boto3 matplotlib 

In [ ]:
#to import modules required otherwise error will arise

import pandas as pd
from readS3 import ReadS3 #reading from readS3
import re

import matplotlib.pyplot as plt

### How to read a file from S3 using readS3.py

In [ ]:
print(ReadS3.__doc__)

In [ ]:
target_file = "s3://alpha-probation-covid-automation/staff_2021-06-29.csv"
bucket_name = "alpha-probation-covid-automation"
s3 = ReadS3(bucket_name)

In [ ]:
#placing item to dataframe
df = s3.read_csv("staff_2021-06-29.csv")

In [ ]:
#columns
df.columns

In [ ]:
#isolating a column
df['date_entry_created']

In [ ]:
df[['date_entry_created','location']]

In [ ]:
#checking the number of blank or None Values
df.isna().sum()

### Quick Analytics

In [ ]:
df['location'].nunique()

In [ ]:
df['location'].unique()

In [ ]:
df['location'].astype(str).str.lower()

In [ ]:
df['location_CLEANED'] = df['location'].astype(str).str.lower()

In [ ]:
df[['location_CLEANED','location']]

In [ ]:
df['date_entry_created'].astype(str).str.replace("[0-9\-\\\]","").unique()

In [ ]:
condition = df['date_entry_created'].fillna("").str.contains("SERCO") 
df[condition]

In [ ]:
# correcting data frame

df.loc[condition,'date_entry_created_DEMO'] = "25/03/2021"

In [ ]:
#creating function

def clean(x):
    if "25/03/20201SERCO" == x:
        return "25/03/2020"
    else:
        return "not required"

clean("25/03/20201SERCO")

In [ ]:
#using lambda to correct data

df['date_entry_created_DEMO_2'] = df.date_entry_created.astype(str).apply(lambda x:clean(x))

In [ ]:
df[condition]

In [ ]:
#### Combining DataFrames

df_brixton = df[df.location.str.strip() == 'Brixton']

df_win = df[df.location.str.strip() == 'Winchester']

#requirement same order of header and same name of header

pd.concat([
    df_brixton,
    df_win
])

### Left Join Vlookup

In [ ]:
df_id = df_brixton[['case_number']]
df_id.merge(df_brixton,how="left",on='case_number')

### Simple Calculations

In [ ]:
df['total_bill'].sum()

In [ ]:
df['total_bill'].cumsum()

In [ ]:
df['total_bill'].mean()

In [ ]:
df['total_bill'].nlargest(10)

### Exporting to S3

In [ ]:
s3.to_csv_S3(df,"test.csv")